# Introduction to CUDA Programming

This tutorial introduces the fundamental concepts of CUDA programming, which is essential for understanding high-performance GPU computing used in LLM inference engines.

## What is CUDA?

CUDA (Compute Unified Device Architecture) is a parallel computing platform and programming model developed by NVIDIA. It allows developers to use NVIDIA GPUs for general-purpose computing tasks beyond graphics rendering.

## Key Concepts

1. **Host vs Device**: The CPU and its memory are referred to as the host, while the GPU and its memory are referred to as the device.
2. **Kernels**: Functions that run on the GPU.
3. **Threads**: The basic unit of execution in CUDA.
4. **Blocks**: Groups of threads that can cooperate and share memory.
5. **Grids**: Collections of blocks.

## Memory Hierarchy

CUDA provides different types of memory with varying scope and performance characteristics:
- **Global Memory**: Large but slow, accessible by all threads
- **Shared Memory**: Fast, shared within a block
- **Registers**: Fastest, private to each thread
- **Constant Memory**: Cached, read-only memory for constants

In [ ]:
import numpy as np
import torch

print("CUDA Basics Demonstration")
print("========================")

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available")
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available on this system")
    print("This is expected since we're running on a simple laptop for demonstration purposes")

## CUDA Thread Hierarchy

Understanding the thread hierarchy is crucial for CUDA programming:

```
Grid (entire computation)
├── Block 0
│   ├── Thread (0,0,0)
│   ├── Thread (0,1,0)
│   ├── Thread (1,0,0)
│   └── Thread (1,1,0)
├── Block 1
│   ├── Thread (0,0,0)
│   ├── Thread (0,1,0)
│   ├── Thread (1,0,0)
│   └── Thread (1,1,0)
└── Block 2
    ├── Thread (0,0,0)
    ├── Thread (0,1,0)
    ├── Thread (1,0,0)
    └── Thread (1,1,0)
```

Each thread can be identified by its unique indices:
- `blockIdx`: Block index within the grid
- `threadIdx`: Thread index within the block
- `blockDim`: Block dimensions
- `gridDim`: Grid dimensions

In [ ]:
# Simulate CUDA thread hierarchy concepts
def simulate_cuda_grid(num_blocks, threads_per_block):
    print(f"Simulating CUDA grid with {num_blocks} blocks, {threads_per_block} threads per block")
    print(f"Total threads: {num_blocks * threads_per_block}")
    
    for block_id in range(num_blocks):
        print(f"\nBlock {block_id}:")
        for thread_id in range(threads_per_block):
            global_thread_id = block_id * threads_per_block + thread_id
            print(f"  Thread {thread_id} (global ID: {global_thread_id})")

# Demonstrate with a small example
simulate_cuda_grid(3, 4)

## Memory Management in CUDA

Memory management is critical in CUDA programming. Data must be explicitly transferred between host (CPU) and device (GPU) memory:

1. **Allocate device memory**
2. **Copy data from host to device**
3. **Execute kernel on device**
4. **Copy results from device to host**
5. **Free device memory**

This process is often referred to as the "CUDA memory management cycle".

In [ ]:
# Simulate CUDA memory management concepts
class CUDAMemorySimulator:
    def __init__(self):
        self.host_memory = {}
        self.device_memory = {}
        self.memory_counter = 0
    
    def malloc_host(self, size, name=None):
        if name is None:
            name = f"host_buffer_{self.memory_counter}"
            self.memory_counter += 1
        
        self.host_memory[name] = np.zeros(size)
        print(f"Allocated {size} elements on host as '{name}'")
        return name
    
    def malloc_device(self, size, name=None):
        if name is None:
            name = f"device_buffer_{self.memory_counter}"
            self.memory_counter += 1
        
        self.device_memory[name] = np.zeros(size)
        print(f"Allocated {size} elements on device as '{name}'")
        return name
    
    def memcpy_host_to_device(self, host_name, device_name):
        if host_name in self.host_memory and device_name in self.device_memory:
            size = min(len(self.host_memory[host_name]), len(self.device_memory[device_name]))
            self.device_memory[device_name][:size] = self.host_memory[host_name][:size]
            print(f"Copied {size} elements from '{host_name}' to '{device_name}'")
    
    def memcpy_device_to_host(self, device_name, host_name):
        if device_name in self.device_memory and host_name in self.host_memory:
            size = min(len(self.device_memory[device_name]), len(self.host_memory[host_name]))
            self.host_memory[host_name][:size] = self.device_memory[device_name][:size]
            print(f"Copied {size} elements from '{device_name}' to '{host_name}'")
    
    def free(self, name):
        if name in self.host_memory:
            del self.host_memory[name]
            print(f"Freed host memory '{name}'")
        elif name in self.device_memory:
            del self.device_memory[name]
            print(f"Freed device memory '{name}'")

# Demonstrate CUDA memory management cycle
print("CUDA Memory Management Cycle Simulation")
print("=====================================")

simulator = CUDAMemorySimulator()

# 1. Allocate host memory
host_data = simulator.malloc_host(10, "input_data")

# Initialize host data
simulator.host_memory[host_data] = np.arange(10, dtype=np.float32)
print(f"Initialized host data: {simulator.host_memory[host_data]}")

# 2. Allocate device memory
device_data = simulator.malloc_device(10, "gpu_buffer")

# 3. Copy data from host to device
simulator.memcpy_host_to_device(host_data, device_data)
print(f"Device data after copy: {simulator.device_memory[device_data]}")

# 4. Simulate kernel execution (square each element)
print("\nSimulating kernel execution (squaring each element)")
simulator.device_memory[device_data] = np.square(simulator.device_memory[device_data])
print(f"Device data after kernel: {simulator.device_memory[device_data]}")

# 5. Copy results back to host
result_data = simulator.malloc_host(10, "output_data")
simulator.memcpy_device_to_host(device_data, result_data)
print(f"Host result data: {simulator.host_memory[result_data]}")

# 6. Free memory
simulator.free(host_data)
simulator.free(device_data)
simulator.free(result_data)

## CUDA Kernel Example

A CUDA kernel is a function that runs on the GPU. Here's a simple example of what a vector addition kernel looks like in CUDA C++:

```cpp
__global__ void vectorAdd(float *A, float *B, float *C, int N)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    
    if (i < N)
        C[i] = A[i] + B[i];
}
```

Key points about this kernel:
1. `__global__`: Indicates this is a kernel function callable from host code
2. `blockDim.x * blockIdx.x + threadIdx.x`: Calculates the global thread index
3. Boundary checking: Ensures threads don't access memory outside the array
4. Parallel execution: Each thread processes one element

In [ ]:
# Simulate a CUDA kernel execution
def simulate_vector_add_kernel(a, b):
    """Simulate a vector addition kernel execution"""
    n = len(a)
    c = np.zeros(n)
    
    print(f"Simulating vector addition of {n} elements")
    print(f"A: {a}")
    print(f"B: {b}")
    
    # Simulate parallel execution with threads
    threads_per_block = 4
    num_blocks = (n + threads_per_block - 1) // threads_per_block
    
    print(f"\nLaunching kernel with {num_blocks} blocks, {threads_per_block} threads per block")
    
    for block_id in range(num_blocks):
        print(f"  Block {block_id}:")
        for thread_id in range(threads_per_block):
            global_id = block_id * threads_per_block + thread_id
            
            if global_id < n:
                c[global_id] = a[global_id] + b[global_id]
                print(f"    Thread {thread_id}: C[{global_id}] = A[{global_id}] + B[{global_id}] = {a[global_id]} + {b[global_id]} = {c[global_id]}")
    
    return c

# Demonstrate kernel execution
a = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=np.float32)
b = np.array([10, 20, 30, 40, 50, 60, 70, 80], dtype=np.float32)
c = simulate_vector_add_kernel(a, b)
print(f"\nResult: {c}")

## Performance Considerations

When writing CUDA code, several performance factors need to be considered:

1. **Memory Coalescing**: Ensuring threads in a warp access contiguous memory locations
2. **Occupancy**: Maximizing the number of active warps per SM
3. **Divergence**: Avoiding conditional branches that cause threads in a warp to take different paths
4. **Shared Memory Usage**: Efficiently using shared memory to reduce global memory accesses

## Summary

CUDA programming involves understanding:
- The parallel execution model (grids, blocks, threads)
- Memory management between host and device
- Kernel design for parallel execution
- Performance optimization techniques

These concepts form the foundation for implementing high-performance GPU kernels used in LLM inference engines like vLLM.